# Pipelines e transformadores compostos

## Transformadores

A biblioteca Scikit-Learn tem vários componentes chamados **Transformadores** (Transformers) para transformar os dados. Um transformador é um objeto de uma classe que implemente os seguintes métodos:

- `fit(X_train, y_train = None)`
    - Recebe as *features* `X_train` e "treina" o transformador. 
    - Recebe também opcionalmente o *target* `y_train` por compatibilidade com *estimadores* (e.g. modelos), mas não usa.
- `transform(X_test)`
    - Recebe um novo conjunto de *features* `X_test` e aplica a transformação aprendida (no passo `fit`) aos dados.

Vamos ver alguns exemplos de transformadores do Scikit-Learn:

### `StandardScaler`

O `sklearn.preprocessing.StandardScaler` realiza a normalização de cada coluna do dataset:

- No método `fit` calcula e armazena a média e o desvio padrão de cada coluna do dataset.
- No método `transform` aplica a cada coluna a fórmula

$$
z = \frac{(x - m)}{s}
$$

onde $m$ é o valor médio da coluna e $s$ é o desvio padrão da coluna. Vamos ver um exemplo:

In [1]:
# Vou usar o magic %reset -f para limpar as variáveis do kernel antes de
# cada exemplo, para evitar problemas de variáveis globais e garantir
# que o exemplo é autocontido.
#
# Estou usando a versão sem "%" para não ter problemas com o linting.
from IPython import get_ipython, display

get_ipython().run_line_magic('reset', '-f')

In [2]:
import numpy as np

# Faz uns dados falsos para testar o StandardScaler.
X_train = np.array(
    [
        [1, 2],
        [3, 4],
        [5, 6],
        [7, 8],
    ],
    dtype=np.float64,
)
print(X_train)

[[1. 2.]
 [3. 4.]
 [5. 6.]
 [7. 8.]]


In [3]:
# Mostra a média de cada coluna dos dados originais.
print(X_train.mean(axis=0))

[4. 5.]


In [4]:
# Mostra o desvio padrão de cada coluna dos dados originais.
print(X_train.std(axis=0))

[2.23606798 2.23606798]


Agora vamos criar e treinar um `StandardScaler`:

In [5]:
from sklearn.preprocessing import StandardScaler

# Cria um scaler.
scaler = StandardScaler()

# Mostra o scaler antes de treinar, vai aparecer laranja indicando que não foi
# treinado ainda.
display(scaler)

StandardScaler()

In [6]:
# Treina o scaler nos dados.
scaler.fit(X_train)

# Mostra o scaler depois de treinar, vai aparecer azul indicando
# que foi treinado.
display(scaler)

StandardScaler()

O `StandardScaler` aprendeu a média e o desvio padrão de cada coluna do conjunto de treinamento. Essa informação pode ser vista nos *atributos* `mean_` e `scale_`:

In [7]:
print(scaler.mean_)

[4. 5.]


In [8]:
print(scaler.scale_)

[2.23606798 2.23606798]


Compare com os valores de média e desvio padrão fornecidos pelo `numpy` acima: são idênticos.

Vamos transformar os próprios dados de treinamento com o `StandardScaler`. O que se espera aqui? Que os dados transformados tenham média zero e desvio padrão $1$.

In [9]:
# Usa o scaler para transformar os dados.
X_train_scaled = scaler.transform(X_train)

print(X_train_scaled)

[[-1.34164079 -1.34164079]
 [-0.4472136  -0.4472136 ]
 [ 0.4472136   0.4472136 ]
 [ 1.34164079  1.34164079]]


In [10]:
# Mostra a média de cada coluna dos dados transformados.
print(np.mean(X_train_scaled, axis=0))

[0. 0.]


In [11]:
# Mostra o desvio padrão de cada coluna dos dados transformados.
print(np.std(X_train_scaled, axis=0))

[1. 1.]


Note que aplicar a transformação do `scaler` aos mesmos dados em que foi treinado resulta em um *dataset* com média zero e desvio padrão $1$ em cada coluna, como esperado. A aplicação de `fit` seguido de `transform` no mesmo *dataset* pode ser feita de uma vez só com o método `fit_transform`:

In [12]:
X_train_scaled = scaler.fit_transform(X_train)

print(X_train_scaled)

[[-1.34164079 -1.34164079]
 [-0.4472136  -0.4472136 ]
 [ 0.4472136   0.4472136 ]
 [ 1.34164079  1.34164079]]


Agora vamos testar o `scaler` já treinado em outro *dataset*:

In [13]:
X_test = np.array(
    [
        [3, 1],
        [4, 1],
        [5, 9],
        [2, 6],
        [5, 3],
    ],
    dtype=np.float64,
)

print(X_test.mean(axis=0))
print(X_test.std(axis=0))

[3.8 4. ]
[1.16619038 3.09838668]


In [14]:
X_test_scaled = scaler.transform(X_test)

print(X_test_scaled)

[[-0.4472136  -1.78885438]
 [ 0.         -1.78885438]
 [ 0.4472136   1.78885438]
 [-0.89442719  0.4472136 ]
 [ 0.4472136  -0.89442719]]


In [15]:
print(X_test_scaled.mean(axis=0))
print(X_test_scaled.std(axis=0))

[-0.08944272 -0.4472136 ]
[0.52153619 1.38564065]


Observe que neste novo *dataset* a aplicação do `StandardScaler` já treinado **não** resulta em um *dataset* de média zero e desvio padrão $1$. Afinal, o `StandardScaler` não foi treinado para este *dataset*, foi treinado em outro *dataset*. Logo, esse resultado está correto, é o esperado mesmo!

### `SimpleImputer` e outros *imputers*

> *Impute*: to attribute or ascribe.

Um dos significados do verbo "to impute" em inglês é atribuir algo a alguém ou alguma coisa. Em Ciência dos Dados, significa atribuir valor a células vazias por algum método. Ou seja: preencher os buracos.

O Scikit-Learn tem alguns transformadores para auxiliar nesta tarefa. O *imputer* mais simples deles chama-se... `SimpleImputer` (criativo, não?).

In [16]:
from IPython import get_ipython

get_ipython().run_line_magic('reset', '-f')

In [17]:
import numpy as np

# Fazendo um dataset para testar.
X_train = np.array(
    [
        [1, 2],
        [None, 4],
        [1, None],
        [1, None],
    ],
    dtype=np.float64,
)

In [18]:
from sklearn.impute import SimpleImputer

# Cria um imputer.
imputer = SimpleImputer()

imputer

SimpleImputer()

In [19]:
# Treina o imputer nos dados.
imputer.fit(X_train)

imputer

SimpleImputer()

No seu modo padrão, o `SimpleImputer` aprende a média de cada coluna para preencher os buracos:

In [20]:
imputer.statistics_

array([1., 3.])

Agora, ao aplicar o método `transform` do `SimpleImputer` a um *dataset*, quaisquer buracos serão preenchidos com os valores aprendidos da respectiva coluna:

In [21]:
X_test = np.array([
    [None, 1],
    [2, None],
    [None, None],
])

X_test_imputed = imputer.transform(X_test)

X_test_imputed

array([[1., 1.],
       [2., 3.],
       [1., 3.]])

Observe o preenchimento dos buracos.

Caso este *imputer* simples não seja suficiente, o Scikit-Learn possui outros *imputers* no pacote `sklearn.impute`, verifique a documentação!

### `OneHotEncoder`

O `OneHotEncoder` cria variáveis *dummy* para colunas categóricas, veja o exemplo:

In [22]:
from IPython import get_ipython

get_ipython().run_line_magic('reset', '-f')

In [23]:
import numpy as np

# Fazendo um dataset para testar.
X_train = np.array([
    ['apple'],
    ['papaya'],
    ['coconut'],
    ['banana'],
    ['apple'],
    ['banana'],
],)

X_train

array([['apple'],
       ['papaya'],
       ['coconut'],
       ['banana'],
       ['apple'],
       ['banana']], dtype='<U7')

In [24]:
from sklearn.preprocessing import OneHotEncoder

# Cria um encoder.
# O argumento sparse_output=False faz com que o encoder retorne um array
# numpy ao invés de uma matriz esparsa.
encoder = OneHotEncoder(sparse_output=False)

encoder

OneHotEncoder(sparse_output=False)

In [25]:
encoder.fit(X_train)

encoder

OneHotEncoder(sparse_output=False)

In [26]:
encoder.categories_

[array(['apple', 'banana', 'coconut', 'papaya'], dtype='<U7')]

In [27]:
X_train_encoded = encoder.transform(X_train)

X_train_encoded

array([[1., 0., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 1., 0.],
       [0., 1., 0., 0.],
       [1., 0., 0., 0.],
       [0., 1., 0., 0.]])

Note que temos um número de colunas igual ao número de categorias diferentes no *dataset* de treino, e que em cada linha todos os valores são zero, exceto por um valor $1$ na coluna cujo indice corresponde ao indice da categoria vista no *dataset*. Ou seja:

- Na primeira linha a categoria vista no *dataset* era "apple", e "apple" é a primeira categoria da lista de categorias "aprendida" pelo encoder. Portanto a primeira linha do *encoded dataset* tem $1$ na primeira coluna e zero nas demais.
- Na segunda linha a categoria é "papaya", que é a quarta categoria do encoder, logo a linha do *encoded dataset* tem $1$ na quarta coluna e zero nas demais.
- e assim por diante

In [28]:
# Cria um dataset de teste.
X_test = np.array([
    ['banana'],
    ['apple'],
    ['papaya'],
    ['coconut'],
],)

X_test_encoded = encoder.transform(X_test)

X_test_encoded

array([[0., 1., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 1., 0.]])

Como esperado. E se tiver uma classe desconhecida no *dataset* de teste?

In [29]:
X_test_unsafe = np.array([
    ['banana'],
    ['apple'],
    ['papaya'],
    ['coconut'],
    ['strawberry'],
],)

try:
    X_test_unsafe_encoded = encoder.transform(X_test_unsafe)
except ValueError as e:
    print(f'ValueError: {e}')


ValueError: Found unknown categories [np.str_('strawberry')] in column 0 during transform


Veja que a aplicação de `transform` em um *dataset* que contém categorias inéditas resulta em uma exceção do tipo `ValueError`.

Para fazer com que o `OneHotEncoder` não lance um erro nestes casos, crie o `OneHotEncoder` com o parâmetro `handle_unknown='ignore'`:

In [30]:
safe_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

safe_encoder.fit(X_train)

X_test_unsafe_encoded = safe_encoder.transform(X_test_unsafe)

X_test_unsafe_encoded

array([[0., 1., 0., 0.],
       [1., 0., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 1., 0.],
       [0., 0., 0., 0.]])

Observe que:

- Não temos mais erros.
- Ao encontrar uma categoria inédita, o `OneHotEncoder` simplesmente não atribui o valor $1$ a nenhuma coluna: veja o que aconteceu com a ultima linha de `X_test_unsafe_encoded`.

Certos modelos requerem que a saída do `OneHotEncoder` tenha uma coluna a menos que o número de categorias descobertas no *dataset* de treinamento. Por exemplo, modelos lineares sem regularização. Neste caso, podemos criar o `OneHotEncoder` com o parâmetro `drop='first'`:

In [31]:
encoder = OneHotEncoder(
    sparse_output=False,
    drop='first',
    handle_unknown='ignore',
)

X_train_encoded = encoder.fit_transform(X_train)

X_train_encoded

array([[0., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 0.],
       [1., 0., 0.]])

In [32]:
encoder.categories_

[array(['apple', 'banana', 'coconut', 'papaya'], dtype='<U7')]

Note que o *encoder* aprendeu corretamente sobre a existência de $4$ categorias, mas a saída do método `transform` tem apenas $3$ colunas. A primeira linha de `X_train` apresentava o valor `apple`, que é a primeira categoria da lista de categorias aprendidas: consequentemente a primeira linha do resultado da codificação *one-hot* com parâmetro `drop='first'` tem apenas zeros.

### PolynomialFeatures

O transformador `PolynomialFeatures` cria novas *features* a partir das existentes pela aplicação de *monômios* combinando as várias colunas. O grau dos monômios é escolhido no momento de criação do transformador. Vamos ilustrar este transformador com um exemplo:

In [33]:
from IPython import get_ipython

get_ipython().run_line_magic('reset', '-f')

In [34]:
import numpy as np

# Fazendo um dataset para testar.
X_train = np.array(
    [
        [1, 2],
        [3, 4],
        [5, 6],
    ],
    dtype=np.float64,
)

X_train

array([[1., 2.],
       [3., 4.],
       [5., 6.]])

In [35]:
# PolynomialFeatures de grau 2.
from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures(degree=2)

poly.fit(X_train)

poly.powers_

array([[0, 0],
       [1, 0],
       [0, 1],
       [2, 0],
       [1, 1],
       [0, 2]])

Ok, muita informação aqui. O *array* `poly.powers_` tem $6$ linhas e $2$ colunas, e está preenchido com os inteiros $0$, $1$ ou $2$. Isso significa que a saída do método `transform` terá $6$ colunas, uma para cada linha de `poly.powers_`, e que cada uma dessas colunas terá um *monômio* feito das *features* do *dataset* elevadas ao grau indicado na linha de `poly.powers_`.

Traduzindo:

- Vamos tentar entender o que acontecerá ao aplicar o método `transform` a um *dataset* `X_test`
- Para uma linha qualquer do *dataset* `X_test` vamos chamar de $a$ o valor na primeira coluna e $b$ o valor na segunda coluna
- No resultado teremos $6$ colunas:
    - Na primeira coluna teremos $a^0b^0 = 1$, termo de grau zero.
    - Na segunda coluna teremos $a^1b^0 = a$, termo de grau um.
    - Na terceira coluna teremos $a^0b^1 = b$, termo de grau um.
    - Na quarta coluna teremos $a^2b^0 = a^2$, termo de grau dois.
    - Na quinta coluna teremos $a^1b^1 = ab$, termo de grau dois.
    - Na sexta coluna teremos $a^0b^2 = b^2$, termo de grau dois.

Vamos verificar isso:

In [36]:
X_test = np.array(
    [
        [7, 8],
        [9, 10],
    ],
    dtype=np.float64,
)

X_test_poly = poly.transform(X_test)

X_test_poly

array([[  1.,   7.,   8.,  49.,  56.,  64.],
       [  1.,   9.,  10.,  81.,  90., 100.]])

Verifique que o resultado bate com a explicação acima.

Para alguns modelos, a adição da primeira coluna feita de $1$s é indesejável. Novamente, o exemplo principal aqui é o modelo linear, que calha de ser o modelo mais popular de todos e portanto não pode ser ignorado. Para gerar *features* polinomiais sem a coluna de valores $1$ use o parâmetro `include_bias=False`

In [37]:
poly = PolynomialFeatures(degree=2, include_bias=False)

poly.fit(X_train)

poly.powers_

array([[1, 0],
       [0, 1],
       [2, 0],
       [1, 1],
       [0, 2]])

Nada de termo de grau zero agora. Vejamos como fica a saída do método `transform`:

In [38]:
X_test_poly = poly.transform(X_test)

X_test_poly

array([[  7.,   8.,  49.,  56.,  64.],
       [  9.,  10.,  81.,  90., 100.]])

Nada de coluna de $1$

### `FunctionTransformer`

O Scikit-Learn já tem vários transformadores prontos para serem usados em seus pipelines, consulte a documentação para mais detalhes. Mas e se você precisar de um transformador que não está disponível no Scikit-Learn? Uma boa opção é usar o `FunctionTransformer`, que te permite especificar uma função a ser aplicada ao seu *dataset*:

In [39]:
from IPython import get_ipython

get_ipython().run_line_magic('reset', '-f')

In [40]:
import numpy as np

# Fazendo um dataset para testar.
X_train = np.array(
    [
        [1, 2],
        [3, 4],
        [5, 6],
    ],
    dtype=np.float64,
)

X_train

array([[1., 2.],
       [3., 4.],
       [5., 6.]])

Vamos construir um transformador que calcula a soma dos quadrados das *features* e cria uma nova *feature*:

In [41]:
def add_sum_squared_feature(X: np.ndarray) -> np.ndarray:
    ''' Adiciona uma coluna com a soma dos quadrados das colunas de X. '''

    def sum_squared(X: np.ndarray) -> np.ndarray:
        ''' Retorna a soma dos quadrados das colunas de X. '''
        return np.sum(X**2, axis=1).reshape(-1, 1)

    return np.concatenate([X, sum_squared(X)], axis=1)


add_sum_squared_feature(X_train)

array([[ 1.,  2.,  5.],
       [ 3.,  4., 25.],
       [ 5.,  6., 61.]])

In [42]:
from sklearn.preprocessing import FunctionTransformer

transformer = FunctionTransformer(add_sum_squared_feature)

transformer

FunctionTransformer(func=<function add_sum_squared_feature at 0x7fc0067a60c0>)

Observe que o transformador já vem "treinado", como assim? Isso acontece porque o `FunctionTransformer` só pode aplicar transformações **sem estado**, ou seja, que não requerem treinamento, só fazer a aplicação de uma função.

Mesmo assim, o transformador tem o método `fit`, que só faz uma verificação de dados. Nesta verificação o transformador pelo menos guarda o número de *features* do *dataset* de treinamento, para que ao aplicar o método `transform` em outro *dataset* não exista conflito de número de colunas.

In [43]:
transformer.fit(X_train)

transformer.n_features_in_

2

Agora vamos ver o transformador em ação:

In [44]:
# Fazendo um dataset de teste.
X_test = np.array(
    [
        [7, 8],
        [9, 10],
    ],
    dtype=np.float64,
)

X_test_transformed = transformer.transform(X_test)

X_test_transformed

array([[  7.,   8., 113.],
       [  9.,  10., 181.]])

O `FunctionTransformer` trabalha com qualquer tipo de dados, na verdade (desde que o parâmetro de criação `validate` seja `False`, verifique a documentação). Vamos testar um `FunctionTransformer` para trocar uma coluna de texto pelo texto reverso, e ainda adicionar uma coluna indicando se o texto era *palíndromo* (leitura idêntica nos dois sentidos):

In [45]:
from functools import partial


@partial(np.frompyfunc, nin=1, nout=1)
def reverse_text(text: str) -> str:
    ''' Inverte o texto. '''
    return text[::-1]


@partial(np.frompyfunc, nin=1, nout=1)
def is_palindrome(text: str) -> bool:
    ''' Retorna True se o texto é um palíndromo. '''
    return text == text[::-1]


def reverse_text_and_check_palindrome(X: np.array) -> np.array:
    ''' Inverte o texto de cada linha de X e adiciona uma coluna indicando
    se o texto original é um palíndromo. '''
    reversed_texts = np.apply_along_axis(reverse_text, axis=1, arr=X)
    palindromes = np.apply_along_axis(is_palindrome, axis=1, arr=X)
    return np.column_stack([reversed_texts, palindromes])

In [46]:
transformer = FunctionTransformer(reverse_text_and_check_palindrome)

transformer

FunctionTransformer(func=<function reverse_text_and_check_palindrome at 0x7fc005948ae0>)

In [47]:
X_train = np.array([
    [
        'ana',
        'banana',
        'arara',
        'casa',
        'Insper',
        'Python',
        'radar',
        'roma é amor',
    ],
    [
        'anna',
        'banana',
        'macaw',
        'house',
        'Insper',  # The one and only!
        'Python',  # Same!
        'radar',
        'roma is love',
    ],
]).transpose()

X_train

array([['ana', 'anna'],
       ['banana', 'banana'],
       ['arara', 'macaw'],
       ['casa', 'house'],
       ['Insper', 'Insper'],
       ['Python', 'Python'],
       ['radar', 'radar'],
       ['roma é amor', 'roma is love']], dtype='<U12')

In [48]:
X_train_transformed = transformer.fit_transform(X_train)

X_train_transformed

array([['ana', 'anna', True, True],
       ['ananab', 'ananab', False, False],
       ['arara', 'wacam', True, False],
       ['asac', 'esuoh', False, False],
       ['repsnI', 'repsnI', False, False],
       ['nohtyP', 'nohtyP', False, False],
       ['radar', 'radar', True, True],
       ['roma é amor', 'evol si amor', True, False]], dtype=object)

Vamos ver mais a seguir (seção "Trabalhando com `DataFrame`") que as versões mais recentes do Scikit-Learn trabalham bem com `DataFrame` do Pandas, mantendo o nome das colunas, etc. No caso do `FunctionTransformer`, se você já retorna um `DataFrame`, nem precisa se preocupar.

In [49]:
import pandas as pd

X_train_df = pd.DataFrame({
    'word': [
        'ana',
        'banana',
        'arara',
        'casa',
        'Insper',
        'Python',
        'radar',
        'roma é amor',
    ],
    'word_in_english': [
        'anna',
        'banana',
        'macaw',
        'house',
        'Insper',  # The one and only!
        'Python',  # Same!
        'radar',
        'roma is love',
    ]
})

X_train_df

,word,word_in_english
0,ana,anna
1,banana,banana
2,arara,macaw
3,casa,house
4,Insper,Insper
5,Python,Python
6,radar,radar
7,roma é amor,roma is love


In [50]:
def reverse_text_and_check_palindrome_df(X: pd.DataFrame) -> pd.DataFrame:
    ''' Inverte o texto de cada linha de X e adiciona uma coluna indicando
    se o texto original é um palíndromo.
    '''
    reversed_texts = X.map(reverse_text)
    reversed_texts.columns = [
        f'reversed_{col}' for col in reversed_texts.columns
    ]

    palindromes = X.map(is_palindrome)
    palindromes.columns = [
        f'is_{col}_palindrome' for col in palindromes.columns
    ]

    return pd.concat([reversed_texts, palindromes], axis=1)


transformer = FunctionTransformer(reverse_text_and_check_palindrome_df)

In [51]:
X_train_df_transformed = transformer.fit_transform(X_train_df)

X_train_df_transformed

,reversed_word,reversed_word_in_english,is_word_palindrome,is_word_in_english_palindrome
0,ana,anna,True,True
1,ananab,ananab,False,False
2,arara,wacam,True,False
3,asac,esuoh,False,False
4,repsnI,repsnI,False,False
5,nohtyP,nohtyP,False,False
6,radar,radar,True,True
7,roma é amor,evol si amor,True,False


Mas uma prática mais interessante é deixar o `FunctionTransformer` criar os nomes de colunas para você. Com isso você poderá trabalhar tanto com `DataFrames` como com arrays do Numpy.

Para ter esse efeito em um `FunctionTransformer` você precisa:

- Implementar uma função que recebe dois argumentos: 
    - `self`, indicando o próprio `FunctionTransformer`, e 
    - `input_features`, que são os nomes das *features* de entrada.
    - A função então retorna o nome das *features* de saída.
- A função de transformação deve retornar um array do Numpy apenas.
- Você deve chamar o método `set_output(transform='pandas')` no seu transformador antes de usá-lo (ver mais sobre isso na seção "Trabalhando com `DataFrame`")

Vamos ver isso no exemplo do palíndromo:

In [52]:
def reverse_text_and_check_palindrome_df_or_array(
        X: pd.DataFrame | np.ndarray) -> np.ndarray:
    ''' Inverte o texto de cada linha de X e adiciona uma coluna indicando
    se o texto original é um palíndromo. Funciona com dataframes ou arrays.
    '''
    if isinstance(X, pd.DataFrame):
        X = X.values

    return reverse_text_and_check_palindrome(X)


def feature_names_out(
    self: FunctionTransformer,
    input_features: list[str],
) -> list[str]:
    ''' Retorna os nomes das colunas de X após a transformação. '''
    output_features = []

    for name in input_features:
        output_features.append(f'reversed_{name}')

    for name in input_features:
        output_features.append(f'is_palindrome_{name}')

    return output_features

In [53]:
transformer = FunctionTransformer(
    reverse_text_and_check_palindrome_df_or_array,
    feature_names_out=feature_names_out,
)

transformer.set_output(transform='pandas')

FunctionTransformer(feature_names_out=<function feature_names_out at 0x7fc005651080>,
                    func=<function reverse_text_and_check_palindrome_df_or_array at 0x7fc0056511c0>)

In [54]:
# Funciona com np.ndarray!
X_train_transformed = transformer.fit_transform(X_train)

X_train_transformed

,reversed_x0,reversed_x1,is_palindrome_x0,is_palindrome_x1
0,ana,anna,True,True
1,ananab,ananab,False,False
2,arara,wacam,True,False
3,asac,esuoh,False,False
4,repsnI,repsnI,False,False
5,nohtyP,nohtyP,False,False
6,radar,radar,True,True
7,roma é amor,evol si amor,True,False


In [55]:
# Funciona com pd.DataFrame!
X_train_df_transformed = transformer.fit_transform(X_train_df)

X_train_df_transformed

,reversed_word,reversed_word_in_english,is_palindrome_word,is_palindrome_word_in_english
0,ana,anna,True,True
1,ananab,ananab,False,False
2,arara,wacam,True,False
3,asac,esuoh,False,False
4,repsnI,repsnI,False,False
5,nohtyP,nohtyP,False,False
6,radar,radar,True,True
7,roma é amor,evol si amor,True,False


### Transformadores customizados

Existem situações em que você quer fazer um transformador customizado, treinável, etc. Nestes casos você pode fazer seu próprio transformador! Basta fazer o seguinte:

1. Crie uma classe para seu transformador, que herda das classes `TransformerMixin` e `BaseEstimator`, **nesta ordem**.

2. Implemente os métodos `fit` e `transform` (ou se for um novo tipo de modelo, implemente `predict`)

3. Implemente também o método `get_feature_names_out`

Vamos fazer um exemplo. Vamos criar um transformador com as seguintes características:

- `__init__`

    - Recebe no construtor os valores
    
        - `flag`: booleano (`True` ou `False`) indicando se devemos criar colunas indicadoras de saturação.
        - `min_quantile` e `max_quantile`: valores de saturação inferior e superior, respectivamente.

- `fit`

    - Calcula, para cada coluna, os valores numéricos correspondentes aos quantis indicados

- `transform`

    - Para cada valor em cada coluna:

        - Se o valor estiver entre os quantis, fica igual
        - Se o valor estiver abaixo do quantil inferior, satura neste valor
        - Se o valor estiver acima do quantil superior, satura neste valor

    - Se o `flag` tiver sido `True`, crie tambem as colunas `{feature}_is_low` e `{feature}_is_high`, indicando que houve saturação naquela *feature* e naquele valor.

- `get_feature_names_out`

    - Retorna a lista de nomes de colunas do `pd.DataFrame` transformado.

A documentação completa sobre como implementar um transformador customizado está em https://scikit-learn.org/stable/developers/develop.html

In [56]:
from IPython import get_ipython

get_ipython().run_line_magic('reset', '-f')

In [57]:
from typing import Self

import numpy as np
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.utils.validation import check_is_fitted


class Clamper(TransformerMixin, BaseEstimator):

    def __init__(
        self,
        min_quantile: float = 0.0,
        max_quantile: float = 1.0,
        flag: bool = False,
    ) -> None:
        self.min_quantile = min_quantile
        self.max_quantile = max_quantile
        self.flag = flag

    def fit(self, X: np.ndarray, y: np.ndarray | None = None) -> Self:
        if isinstance(X, pd.DataFrame):
            X = X.values

        self.min_values_ = np.quantile(X, self.min_quantile, axis=0)
        self.max_values_ = np.quantile(X, self.max_quantile, axis=0)

        return self

    def transform(self, X: np.ndarray | pd.DataFrame) -> np.ndarray:
        check_is_fitted(self)

        if isinstance(X, pd.DataFrame):
            assert X.columns == self.feature_names_in_, 'Columns do not match'
            X = X.values

        X_clipped = X.copy()

        X_is_low = X < self.min_values_
        X_clipped = np.where(X_is_low, self.min_values_, X_clipped)

        X_is_high = X > self.max_values_
        X_clipped = np.where(X_is_high, self.max_values_, X_clipped)

        if self.flag:
            X_clipped = np.concatenate([X, X_is_low, X_is_high], axis=1)

        return X

    def get_feature_names_out(
        self,
        input_features: list[str] | None = None,
    ) -> list[str]:
        check_is_fitted(self)

        output_features = [f'{name}_clipped' for name in input_features]

        if self.flag:
            output_features += [f'{name}_is_low' for name in input_features]
            output_features += [f'{name}_is_high' for name in input_features]

        return output_features

In [61]:
my_transformer = Clamper(min_quantile=0.3, max_quantile=0.7, flag=True)

my_transformer.set_output(transform='pandas')

X_train = np.concatenate(
    [
        np.linspace(0, 1, 100).reshape(-1, 1),
        np.linspace(0, 1, 100).reshape(-1, 1),
    ],
    axis=1,
)
X_train_df = pd.DataFrame(X_train, columns=['a', 'b'])

my_transformer.fit(X_train)

X_train_clipped = my_transformer.transform(X_train)

X_train_clipped

,0,1
0,0.000000,0.000000
1,0.010101,0.010101
2,0.020202,0.020202
3,0.030303,0.030303
4,0.040404,0.040404
...,...,...
95,0.959596,0.959596
96,0.969697,0.969697
97,0.979798,0.979798
98,0.989899,0.989899


## Trabalhando com `DataFrame`

Todos os exemplos acima usam *arrays* do Numpy. Porém é muito mais conveniente trabalhar com `DataFrames` do Pandas: as colunas tem nome.

Versões mais recentes do Scikit-Learn estão melhorando o tratamento de dados com `DataFrames`. Vamos testar o `StandardScaler` como exemplo:

In [ ]:
from IPython import get_ipython

get_ipython().run_line_magic('reset', '-f')

In [ ]:
import pandas as pd

# Fazendo um dataset para testar.
X_train = pd.DataFrame(
    {
        'A': [1, 2, 3, 4],
        'B': [5, 6, 7, 8],
        'C': [9, 10, 11, 12],
    },)

X_train

In [ ]:
X_test = pd.DataFrame({
    'A': [13, 14],
    'B': [15, 16],
    'C': [17, 18],
},)

X_test

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

scaler.fit(X_train)

X_test_scaled = scaler.transform(X_test)

X_test_scaled

Ok, a informação dos nomes de colunas foi descartada ao que parece. Mas esta informação foi adquirida sim, só não foi usada:

In [ ]:
scaler.feature_names_in_

In [ ]:
scaler.get_feature_names_out()

Portando, parece que o `scaler` teria a capacidade de gerar `DataFrame` do Pandas com nomes corretos de coluna. E isto é possível sim, com o método `set_output(transform='pandas')` do transformador:

In [ ]:
scaler_for_pandas = StandardScaler().set_output(transform='pandas')

In [ ]:
scaler_for_pandas.fit(X_train)

X_test_scaled = scaler_for_pandas.transform(X_test)

X_test_scaled

Note que agora temos um `DataFrame` na saída!

In [ ]:
X_train_np = X_train.to_numpy()
X_test_np = X_test.to_numpy()

X_train_np, X_test_np

E se a entrada não for um `DataFrame`?

In [ ]:
scaler_for_pandas = StandardScaler().set_output(transform='pandas')

scaler_for_pandas.fit(X_train_np)
X_test_np_scaled = scaler_for_pandas.transform(X_test_np)

X_test_np_scaled

Ainda assim funciona!

O que acontece com um transformador que cria novas colunas? Vamos testar o `OneHotEncoder`:

In [ ]:
from IPython import get_ipython

get_ipython().run_line_magic('reset', '-f')

In [ ]:
import pandas as pd

# Fazendo um dataset para testar.
X_train = pd.DataFrame(
    {
        'fruit': ['apple', 'kiwi', 'apple', 'banana'],
        'size': ['small', 'large', 'large', 'small'],
    },)

X_train

In [ ]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(sparse_output=False).set_output(transform='pandas')

X_train_encoded = encoder.fit_transform(X_train)

X_train_encoded

O `OneHotEncoder` criou os nomes a partir do nome original das colunas, e dos nomes de categorias aprendidos!

Para forçar `.set_output(transform='pandas')` automaticamente para todos os transformadores, use a função global `set_config`:

In [ ]:
from IPython import get_ipython

get_ipython().run_line_magic('reset', '-f')

In [ ]:
import pandas as pd
from sklearn import set_config
from sklearn.preprocessing import OneHotEncoder

set_config(transform_output='pandas')

# Fazendo um dataset para testar.

X_train = pd.DataFrame(
    {
        'fruit': ['apple', 'kiwi', 'apple', 'banana'],
        'size': ['small', 'large', 'large', 'small'],
    },)

# Testando o OneHotEncoder com a configuração global.
encoder = OneHotEncoder(sparse_output=False)

X_train_encoded = encoder.fit_transform(X_train)

X_train_encoded

Note que não foi necessário especificar o `set_output(transform='pandas')` para este *encoder*: usamos a configuração global.

## Pipelines

In [ ]:
from IPython import get_ipython

get_ipython().run_line_magic('reset', '-f')

In [ ]:
from sklearn import set_config

set_config(transform_output='pandas')

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures, StandardScaler

In [ ]:
pipe = Pipeline([
    ('imputer', SimpleImputer()),
    ('scaler', StandardScaler()),
    ('poly', PolynomialFeatures()),
])

pipe

In [ ]:
import numpy as np

# Faz uns dados de treino.
X_train = np.array(
    [
        [1, 2],
        [3, None],
        [7, 8],
        [None, 4],
    ],
    dtype=np.float64,
)

y_train = np.array([7, 17, 37, 17], dtype=np.float64)
y_train = y_train + np.random.randn(y_train.shape[0])

# Faz uns dados de teste.
X_test = np.array(
    [
        [5, 6],
        [9, None],
    ],
    dtype=np.float64,
)

y_test = np.array([27, 47], dtype=np.float64)
y_test = y_test + np.random.randn(y_test.shape[0])


In [ ]:
# Treina o pipeline.
pipe.fit(X_train)

pipe

In [ ]:
# Usa o pipeline para transformar os dados de teste.
X_test_transformed = pipe.transform(X_test)

In [ ]:
X_test_transformed

In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression

# Cria uma pipeline com preprocessamento e um modelo de regressão linear.
pipe_with_model = Pipeline([
    ('imputer', SimpleImputer()),
    ('scaler', StandardScaler()),
    ('poly', PolynomialFeatures(degree=2, include_bias=False)),
    ('regressor', LinearRegression()),
])

pipe_with_model

In [ ]:
# Treina o pipeline.
pipe_with_model.fit(X_train, y_train)

pipe_with_model

In [ ]:
# Usa o pipeline para fazer previsões.
y_test_pred = pipe_with_model.predict(X_test)

y_test_pred

In [ ]:
pipe_with_model.named_steps['regressor'].coef_

In [ ]:
pipe_with_model.named_steps['regressor'].intercept_

In [ ]:
features = ['const']
features += list(pipe_with_model['regressor'].feature_names_in_)

coefs = [pipe_with_model['regressor'].intercept_]
coefs += list(pipe_with_model['regressor'].coef_)

In [ ]:
for feature, coef in zip(features, coefs):
    print(f'{feature}: {coef}')